## Review Analysis using RNN

Importing the dataset

In [10]:
from os import listdir
from os.path import isfile, join
positiveFiles = ['positiveReviews/' + f for f in listdir('positiveReviews/') if isfile(join('positiveReviews/', f))]
negativeFiles = ['negativeReviews/' + f for f in listdir('negativeReviews/') if isfile(join('negativeReviews/', f))]
numWords = []
for pf in positiveFiles:
    with open(pf, "r", encoding='utf-8') as f:
        line=f.readline()
        counter = len(line.split())
        numWords.append(counter)       
print('Positive files finished')

for nf in negativeFiles:
    with open(nf, "r", encoding='utf-8') as f:
        line=f.readline()
        counter = len(line.split())
        numWords.append(counter)  
print('Negative files finished')

numFiles = len(numWords)
print('The total number of files is', numFiles)
print('The total number of words in the files is', sum(numWords))
print('The average number of words in the files is', sum(numWords)/len(numWords))

Positive files finished
Negative files finished
The total number of files is 25000
The total number of words in the files is 5844680
The average number of words in the files is 233.7872


In [31]:
positive_reviews = []
negative_reviews = []

for pf in positiveFiles:
    with open(pf, "r", encoding='utf-8') as f:
        line=f.readline()
        positive_reviews.append(line)
print('Positive files finished')

for nf in negativeFiles:
    with open(nf, "r", encoding='utf-8') as f:
        line=f.readline()
        negative_reviews.append(line)
print('Negative files finished')

Positive files finished
Negative files finished


In [32]:
positive = ''
for pf in positive_reviews:
     positive += pf   

In [33]:
negative = ''
for nf in negative_reviews:
     negative += nf   

In [34]:
positive = positive[1:]
negative = negative[1:]

In [35]:
positive[:1000]

'romwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going

In [9]:
print ("Postive Review 1: ", positive.split('\n')[0])
print ('\n')
print ("Negative Review 1: ", negative.split('\n')[0])

Postive Review 1:  ositiveReviews/0_9.txtpositiveReviews/10000_8.txtpositiveReviews/10001_10.txtpositiveReviews/10002_7.txtpositiveReviews/10003_8.txtpositiveReviews/10004_8.txtpositiveReviews/10005_7.txtpositiveReviews/10006_7.txtpositiveReviews/10007_7.txtpositiveReviews/10008_7.txtpositiveReviews/10009_9.txtpositiveReviews/1000_8.txtpositiveReviews/10010_7.txtpositiveReviews/10011_9.txtpositiveReviews/10012_8.txtpositiveReviews/10013_7.txtpositiveReviews/10014_8.txtpositiveReviews/10015_8.txtpositiveReviews/10016_8.txtpositiveReviews/10017_9.txtpositiveReviews/10018_8.txtpositiveReviews/10019_8.txtpositiveReviews/1001_8.txtpositiveReviews/10020_8.txtpositiveReviews/10021_8.txtpositiveReviews/10022_7.txtpositiveReviews/10023_9.txtpositiveReviews/10024_9.txtpositiveReviews/10025_9.txtpositiveReviews/10026_7.txtpositiveReviews/10027_7.txtpositiveReviews/10028_10.txtpositiveReviews/10029_10.txtpositiveReviews/1002_7.txtpositiveReviews/10030_10.txtpositiveReviews/10031_10.txtpositiveRevi

In [62]:
total_words = len(positive.split())
total_characters = len(positive)
unique_words = len(set(positive.split()))
unique_characters = len(set(positive))
total = len(positive.split('\n'))

print ('FOR POSITIVE REVIEWS')
print ("Total words :", total_words)
print ("Total characters :", total_characters)
print ("Unique words :", unique_words)
print ("Unique characters:", unique_characters)
print ('Total: ', total)

FOR POSITIVE REVIEWS
Total words : 2958832
Total characters : 16852002
Unique words : 178599
Unique characters: 168
Total:  12500


In [63]:
total_words = len(negative.split())
total_characters = len(negative)
unique_words = len(set(negative.split()))
unique_characters = len(set(negative))
total = len(negative.split('\n'))

print ('FOR NEGATIVE REVIEWS')
print ("Total words :", total_words)
print ("Total characters :", total_characters)
print ("Unique words :", unique_words)
print ("Unique characters:", unique_characters)
print ('Total: ', total)

FOR NEGATIVE REVIEWS
Total words : 2832310
Total characters : 15997254
Unique words : 174421
Unique characters: 158
Total:  12253


In [69]:
import numpy as np
import tensorflow as tf

pos_labels = ([1] * 12000)
neg_labels = ([0] * 12000)

In [65]:
positive = positive.split('\n')[:12000]
negative = negative.split('\n')[:12000]

In [70]:
reviews = positive + negative
labels = pos_labels + neg_labels

In [74]:
all_text = ' '.join(reviews)
words = all_text.split()

In [82]:
words[:10]

['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy.', 'It', 'ran', 'at', 'the']

In [89]:
from collections import Counter

most_words = Counter()
for x in words:
    most_words[x] +=1

In [98]:
most_words.most_common()[:20]

[('the', 276287),
 ('a', 149085),
 ('and', 146992),
 ('of', 137633),
 ('to', 127415),
 ('is', 99429),
 ('in', 82341),
 ('I', 63264),
 ('that', 62210),
 ('this', 54965),
 ('it', 52305),
 ('/><br', 48947),
 ('was', 44966),
 ('as', 40931),
 ('with', 40086),
 ('for', 39514),
 ('The', 32461),
 ('but', 32447),
 ('on', 29641),
 ('movie', 29265)]

In [91]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

reviews_ints = []
for each in reviews:
    reviews_ints.append([vocab_to_int[word] for word in each.split()])

In [115]:
len(reviews_ints[0])

140

In [114]:
len(reviews[0].split())

140

In [116]:
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 0
Maximum review length: 2470


In [117]:
seq_len = 300
features = np.zeros((len(reviews_ints), seq_len), dtype=int)
for i, row in enumerate(reviews_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]

In [119]:
features[:10,:10]

array([[     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [106968,    906, 106969,     14,    776,  33264, 106970,     38,
            71,     32],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [     0,      0,      0,      0,      0,      0,      0,      0,
             0,      0],
       [   133,     10, 106998,   6469,   1962,   1145,    463,     19,
           319,   4731],
       [   390,  12057, 107014,  25980,  48110,   2698,   9265,   7796,
         27965,   4226],
       [   255,   1358,   2698,   9265,   1248,  13200,    771,      6,
          1337,   1317]])

In [120]:
split_frac = 0.8
split_idx = int(len(features)*0.8)
train_x, val_x = features[:split_idx], features[split_idx:]
train_y, val_y = labels[:split_idx], labels[split_idx:]

test_idx = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_idx], val_x[test_idx:]
val_y, test_y = val_y[:test_idx], val_y[test_idx:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(19200, 300) 
Validation set: 	(2400, 300) 
Test set: 		(2400, 300)


In [121]:
lstm_size = 256
lstm_layers = 1
batch_size = 500
learning_rate = 0.001

In [148]:
n_words = len(vocab_to_int) + 1 # Adding 1 because we use 0's for padding, dictionary started at 1

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None,], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

In [123]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

In [125]:
with graph.as_default():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

In [126]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed,
                                             initial_state=initial_state)

In [127]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [128]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [129]:
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

In [144]:
x, y=next(get_batches(train_x, train_y),1)
len(y)

100

In [150]:
epochs = 10

#with graph.as_default():
#    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y,
                    keep_prob: 0.8,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y,
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

ValueError: Fetch argument <tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros:0' shape=(500, 256) dtype=float32> cannot be interpreted as a Tensor. (Tensor Tensor("MultiRNNCellZeroState/DropoutWrapperZeroState/BasicLSTMCellZeroState/zeros:0", shape=(500, 256), dtype=float32) is not an element of this graph.)